## Setup

In [1]:
from eocanvas import API, Credentials
from eocanvas.api import Input, Config, ConfigOption
from eocanvas.processes import SnapProcess
from eocanvas.snap.graph import Graph

In [2]:
c = Credentials.load()

In [3]:
from hda import Client

In [4]:
c = Client()

## Data

In [5]:
c.metadata("EO:ESA:DAT:SENTINEL-2")

{'type': 'object',
 'title': 'Querable',
 'properties': {'dataset_id': {'title': 'dataset_id',
   'type': 'string',
   'oneOf': [{'const': 'EO:ESA:DAT:SENTINEL-2',
     'title': 'EO:ESA:DAT:SENTINEL-2',
     'group': None}]},
  'bbox': {'title': 'Bbox',
   'type': 'array',
   'minItems': 4,
   'maxItems': 4,
   'items': [{'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90},
    {'type': 'number', 'maximum': 180, 'minimum': -180},
    {'type': 'number', 'maximum': 90, 'minimum': -90}]},
  'productIdentifier': {'title': 'Product Identifier',
   'type': 'string',
   'pattern': '^[a-zA-Z0-9]+$'},
  'productType': {'title': 'Product Type',
   'type': 'string',
   'oneOf': [{'const': 'S2MSI1C', 'title': 'S2MSI1C', 'group': None},
    {'const': 'S2MSI2A', 'title': 'S2MSI2A', 'group': None},
    {'const': 'AUX_GNSSRD', 'title': 'AUX_GNSSRD', 'group': None},
    {'const': 'AUX_PROQUA', 'title': 'AUX_PROQUA', 'group': None},
    {'const': 'AU

In [6]:
q = {
    "dataset_id": "EO:ESA:DAT:SENTINEL-2",
    "startdate": "2020-06-23T00:00:00.000Z",
    "enddate": "2020-06-24T00:00:00.000Z",
    "processingLevel": "S2MSI1C",
    "tileId": "30UUA"
}

In [7]:
r = c.search(q)

In [8]:
url = r.get_download_urls()[0]
inputs = Input(key="img1", url=url)

In [9]:
import json

In [10]:
with open('AOIs.txt', 'r') as fp:
    poly_dict = json.load(fp)

In [11]:
poly_dict

{'2': {'geoRegion': 'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))',
  'tile_Id': '30UUA'},
 '3': {'geoRegion': 'POLYGON ((-4.81917128 50.19269224, -4.59558622 50.19269224, -4.59558622 50.40467681, -4.81917128 50.40467681, -4.81917128 50.19269224))',
  'tile_Id': '30UUA'},
 '4': {'geoRegion': 'POLYGON ((-4.32949749 50.26538565, -4.02885352 50.26538565, -4.02885352 50.5075424, -4.32949749 50.5075424, -4.32949749 50.26538565))',
  'tile_Id': '30UVA'},
 '6': {'geoRegion': 'POLYGON ((-4.02836874 50.14143345, -3.59050258 50.14143345, -3.59050258 50.34954708, -4.02836874 50.34954708, -4.02836874 50.14143345))',
  'tile_Id': '30UVA'},
 '9': {'geoRegion': 'POLYGON ((-3.70217003 50.32171356, -3.4070158 50.32171356, -3.4070158 50.54736093, -3.70217003 50.54736093, -3.70217003 50.32171356))',
  'tile_Id': '30UVA'},
 '11': {'geoRegion': 'POLYGON ((-4.42178593 50.96017888, -3.80799343 50.96017888, -3.80799343 51.

In [12]:
polys_tile0 = {k: v for k, v in poly_dict.items() if v['tile_Id'] == '30UUA'}

In [13]:
poly0 = polys_tile0['2']

In [14]:
poly0['geoRegion']

'POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))'

## Process workflow

In [15]:
#help(SnapProcess)

In [16]:
with open("s2_c2rcc.xml", "r") as fp:
    graph_file = fp.read()

In [17]:
graph_file

'<graph id="Graph">\n  <version>1.0</version>\n  <node id="Read">\n    <operator>Read</operator>\n    <sources/>\n    <parameters class="com.bc.ceres.binding.dom.XppDomElement">\n      <useAdvancedOptions>false</useAdvancedOptions>\n      <file>$img1</file>\n      <copyMetadata>true</copyMetadata>\n      <bandNames/>\n      <pixelRegion>0,0,10980,10980</pixelRegion>\n      <maskNames/>\n    </parameters>\n  </node>\n  <node id="Subset">\n    <operator>Subset</operator>\n    <sources>\n      <sourceProduct refid="Read"/>\n    </sources>\n    <parameters class="com.bc.ceres.binding.dom.XppDomElement">\n      <sourceBands>B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12</sourceBands>\n      <tiePointGrids/>\n      <region>0,0,0,0</region>\n      <referenceBand/>\n      <geoRegion>$polygon</geoRegion>\n      <subSamplingX>1</subSamplingX>\n      <subSamplingY>1</subSamplingY>\n      <fullSwath>false</fullSwath>\n      <copyMetadata>true</copyMetadata>\n    </parameters>\n  </node>\n  <node id="R

In [18]:
modified_xml = graph_file.replace("$polygon", poly0['geoRegion'])

In [19]:
modified_xml

'<graph id="Graph">\n  <version>1.0</version>\n  <node id="Read">\n    <operator>Read</operator>\n    <sources/>\n    <parameters class="com.bc.ceres.binding.dom.XppDomElement">\n      <useAdvancedOptions>false</useAdvancedOptions>\n      <file>$img1</file>\n      <copyMetadata>true</copyMetadata>\n      <bandNames/>\n      <pixelRegion>0,0,10980,10980</pixelRegion>\n      <maskNames/>\n    </parameters>\n  </node>\n  <node id="Subset">\n    <operator>Subset</operator>\n    <sources>\n      <sourceProduct refid="Read"/>\n    </sources>\n    <parameters class="com.bc.ceres.binding.dom.XppDomElement">\n      <sourceBands>B1,B2,B3,B4,B5,B6,B7,B8,B8A,B9,B10,B11,B12</sourceBands>\n      <tiePointGrids/>\n      <region>0,0,0,0</region>\n      <referenceBand/>\n      <geoRegion>POLYGON ((-5.22444257 49.9901843, -4.88225479 49.9901843, -4.88225479 50.29003327, -5.22444257 50.29003327, -5.22444257 49.9901843))</geoRegion>\n      <subSamplingX>1</subSamplingX>\n      <subSamplingY>1</subSampling

In [20]:
graph = Graph.from_text(modified_xml)

In [31]:
help(Config)

Help on class Config in module eocanvas.api:

class Config(builtins.object)
 |  Config(key: 'str', options: 'ConfigOption') -> None
 |  
 |  Config(key: 'str', options: 'ConfigOption')
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, key: 'str', options: 'ConfigOption') -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  asdict(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __annotations__ = {'key': 'str', 'options': 'ConfigOption'}
 |  
 |  __dataclass_fields__ = {'key': Field(name='k

In [21]:
config = Config(key="img1", options=ConfigOption(uncompress=True, sub_path="xfdumanifest.xml"))

In [22]:
process = SnapProcess(snap_graph=graph, eo_config=config, eo_input=inputs)

In [23]:
process.prepare_inputs()['inputs']

{'snap_graph': 'PGdyYXBoIGlkPSJHcmFwaCI+CiAgPHZlcnNpb24+MS4wPC92ZXJzaW9uPgogIDxub2RlIGlkPSJSZWFkIj4KICAgIDxvcGVyYXRvcj5SZWFkPC9vcGVyYXRvcj4KICAgIDxzb3VyY2VzLz4KICAgIDxwYXJhbWV0ZXJzIGNsYXNzPSJjb20uYmMuY2VyZXMuYmluZGluZy5kb20uWHBwRG9tRWxlbWVudCI+CiAgICAgIDx1c2VBZHZhbmNlZE9wdGlvbnM+ZmFsc2U8L3VzZUFkdmFuY2VkT3B0aW9ucz4KICAgICAgPGZpbGU+JGltZzE8L2ZpbGU+CiAgICAgIDxjb3B5TWV0YWRhdGE+dHJ1ZTwvY29weU1ldGFkYXRhPgogICAgICA8YmFuZE5hbWVzLz4KICAgICAgPHBpeGVsUmVnaW9uPjAsMCwxMDk4MCwxMDk4MDwvcGl4ZWxSZWdpb24+CiAgICAgIDxtYXNrTmFtZXMvPgogICAgPC9wYXJhbWV0ZXJzPgogIDwvbm9kZT4KICA8bm9kZSBpZD0iU3Vic2V0Ij4KICAgIDxvcGVyYXRvcj5TdWJzZXQ8L29wZXJhdG9yPgogICAgPHNvdXJjZXM+CiAgICAgIDxzb3VyY2VQcm9kdWN0IHJlZmlkPSJSZWFkIi8+CiAgICA8L3NvdXJjZXM+CiAgICA8cGFyYW1ldGVycyBjbGFzcz0iY29tLmJjLmNlcmVzLmJpbmRpbmcuZG9tLlhwcERvbUVsZW1lbnQiPgogICAgICA8c291cmNlQmFuZHM+QjEsQjIsQjMsQjQsQjUsQjYsQjcsQjgsQjhBLEI5LEIxMCxCMTEsQjEyPC9zb3VyY2VCYW5kcz4KICAgICAgPHRpZVBvaW50R3JpZHMvPgogICAgICA8cmVnaW9uPjAsMCwwLDA8L3JlZ2lvbj4KICAgICAgPHJlZmVyZW5jZUJhbmQv

In [24]:
process.run(download_dir="result")

Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: accepted at 2025-01-23T14:12:20.576829
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:12:30.656180
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:12:41.760440
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:12:54.021314
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:13:07.383469
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:13:22.109789
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:13:38.315305
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: running at 2025-01-23T14:13:56.112945
Job: 7837c673-f8a5-54ec-a77c-c6cec4f3e101 - Status: failed at 2025-01-23T14:14:15.724748


JobFailed: Job 7837c673-f8a5-54ec-a77c-c6cec4f3e101 failed. Try checking the logs for more info.

In [25]:
api = API()

In [26]:
processes = api.get_processes()

In [27]:
processes

[Process(api=<eocanvas.api.API object at 0x7f2a701bc690>, process_id='snap-function', version='10.0.0', title='snap-function', description='Function based on the ESA SNAP tool version 10.', inputs={'eo_config': {'title': 'Configuration for eo_input', 'description': 'Configuration for the eo_input in JSON format. They instruct the function on how to deal with the eo_input (e.g., uncompress).', 'minOccurs': 0, 'maxOccurs': 1, 'schema': {'$ref': 'https://string'}}, 'snap_graph': {'title': 'ESA SNAP GPT graph', 'description': 'A base64-encoded gpt graph. It should contain placeholders referring to the eo_input field.', 'minOccurs': 0, 'maxOccurs': 1, 'schema': {'$ref': 'https://string'}}, 'eo_input': {'title': 'eo_inputs Title', 'description': "Named inputs provided to the ESA SNAP tool in JSON encoding. They should match the placeholders in the SNAP's GPT graph.", 'minOccurs': 0, 'maxOccurs': 1, 'schema': {'$ref': 'http://named_inputs'}}}, output=None),
 Process(api=<eocanvas.api.API obje

In [28]:
jobs = api.get_jobs()
jobs

[Job(api=<eocanvas.api.API object at 0x7f2a701bc690>, job_id='7837c673-f8a5-54ec-a77c-c6cec4f3e101', status='failed', started='2025-01-23 14:12:20', created='2025-01-23 14:12:20', updated='2025-01-23 14:14:07', finished='2025-01-23 14:14:06'),
 Job(api=<eocanvas.api.API object at 0x7f2a701bc690>, job_id='55a62b44-43c9-5375-84db-e2653da4fc8b', status='failed', started='2025-01-23 14:11:27', created='2025-01-23 14:11:27', updated='2025-01-23 14:13:00', finished='2025-01-23 14:12:59'),
 Job(api=<eocanvas.api.API object at 0x7f2a701bc690>, job_id='bf3d51d1-f271-542b-921c-8d86ac6bc167', status='failed', started='2025-01-23 14:02:27', created='2025-01-23 14:02:27', updated='2025-01-23 14:04:01', finished='2025-01-23 14:04:00'),
 Job(api=<eocanvas.api.API object at 0x7f2a701bc690>, job_id='6f8a0bd5-a7ea-56ec-9b3f-b21ce2e81168', status='failed', started='2025-01-22 14:41:09', created='2025-01-22 14:41:09', updated='2025-01-22 14:42:30', finished='2025-01-22 14:42:29'),
 Job(api=<eocanvas.api.A

In [29]:
job = '7837c673-f8a5-54ec-a77c-c6cec4f3e101'

In [30]:
api.get_job_logs(job=job)

[LogEntry(timestamp=datetime.datetime(2025, 1, 23, 14, 12, 40, 926777, tzinfo=datetime.timezone.utc), message='time="2025-01-23T14:12:40.926Z" level=info msg="Starting Workflow Executor" version=v3.5.7'),
 LogEntry(timestamp=datetime.datetime(2025, 1, 23, 14, 12, 40, 931104, tzinfo=datetime.timezone.utc), message='time="2025-01-23T14:12:40.931Z" level=info msg="Using executor retry strategy" Duration=1s Factor=1.6 Jitter=0.5 Steps=5'),
 LogEntry(timestamp=datetime.datetime(2025, 1, 23, 14, 12, 40, 931118, tzinfo=datetime.timezone.utc), message='time="2025-01-23T14:12:40.931Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=ws-serverless podName=workflow-xpd82-stage-in-1769066417 templateName=stage-in version="&Version{Version:v3.5.7,BuildDate:2024-05-27T06:18:59Z,GitCommit:503eef1357ebc9facc3f463708031441072ef7c2,GitTag:v3.5.7,GitTreeState:clean,GoVersion:go1.21.10,Compiler:gc,Platform:linux/amd64,}"'),
 LogEntry(timest

In [32]:
help(api)

Help on API in module eocanvas.api object:

class API(builtins.object)
 |  API(*args, **kwargs)
 |  
 |  Models the Serverless Functions API endpoints.
 |  
 |  Attributes:
 |      urls: The :class:`eocanvas.config.URLs` object that maps all the API endpoints
 |      credentials: A :class:`eocanvas.auth.Credentials` object with username and password
 |  
 |  Methods defined here:
 |  
 |  __init__(self, urls: 'Optional[URLs]' = None, credentials: 'Optional[Credentials]' = None, log_level: 'int' = 20)
 |  
 |  create_key(self, key: 'Key') -> 'Key'
 |  
 |  delete_key(self, key_id: 'str') -> 'None'
 |  
 |  download_result(self, result: 'Result', download_dir: 'Optional[str]' = None)
 |  
 |  exec_process(self, process: 'Process') -> 'Job'
 |      Submits a process to the API.
 |      
 |      Returns:
 |          A :class:`eocanvas.api.Job` instance.
 |  
 |  get_job(self, job_id) -> 'Job'
 |      Gets the details of a job.
 |      
 |      Returns:
 |          A :class:`eocanvas.api.Jo

In [35]:
import eocanvas

In [37]:
help(eocanvas.processes)

Help on module eocanvas.processes in eocanvas:

NAME
    eocanvas.processes

CLASSES
    builtins.object
        ShearWaterParams
            ShearWaterProcess(eocanvas.api.Process, ShearWaterParams)
    eocanvas.api.Process(builtins.object)
        DataTailorProcess(eocanvas.api.Process, _DataTailorParams)
        ShearWaterProcess(eocanvas.api.Process, ShearWaterParams)
        SnapProcess(eocanvas.api.Process, _SnapParams)
    _DataTailorParams(builtins.object)
        DataTailorProcess(eocanvas.api.Process, _DataTailorParams)
    _SnapParams(builtins.object)
        SnapProcess(eocanvas.api.Process, _SnapParams)
    
    class DataTailorProcess(eocanvas.api.Process, _DataTailorParams)
     |  DataTailorProcess(epct_chain: eocanvas.datatailor.chain.Chain, epct_input: List[eocanvas.api.Input] = <factory>, epct_config: List[eocanvas.api.Config] = <factory>, api: 'Optional[API]' = None, process_id: 'Optional[str]' = None, version: 'Optional[str]' = None, title: 'Optional[str]' = None, 